In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)

import numpy as np


from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier



In [2]:
data = pd.read_csv('final_wec_data.csv')

/tmp/ipykernel_1344/3317488320.py:1: DtypeWarning: Columns (26,31,49) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('final_wec_data.csv')


In [3]:
def df_for_ml(df):

    df = df.drop('Unnamed: 0', axis=1)
    df = data[data['year'] >= 2017].reset_index(drop=True)
    # Dropping data from prior to 2017 as driver_ratings data was not available
    df = df.drop(['lap_number', 'car_number', 'lap_number', 'driver_number', 'lap_time', 'elapsed', 'hour', 's1_large', 's2_large', 's3_large', 'driver_name', 'pit_time', 'group', 'team', 'manufacturer', 'season', 'vehicle', 'team_no','lap_time_ms', 'engine', 'driver_stint',
           'team_stint', 'team_stint_no', 'interval_ms', 'interval', 'elapsed_ms', 'position', 'gap', 'elapsed_s'], axis=1)
    # Initial columns dropped because not needed/not useable for modeling. Remaining columns will be further assessed 
    
    return df




In [4]:
data = df_for_ml(data)

In [5]:
def fixing_kept_cols(df):
    df['crossing_finish_line_in_pit'] = df['crossing_finish_line_in_pit'].fillna(0)
    df['crossing_finish_line_in_pit'] = df['crossing_finish_line_in_pit'].replace('B', 1)
#   B represents a car crossing the start/finish line in pitlane, NaN means the car was on track. Values are replaced with binary 0 for on-track, 1 for in-pit

    df = df[df['class']!='INNOVATIVE CAR']
#     removing the innovative class to eliminate outlier data
    
    
    df['circuit'] = df['circuit'].replace({'SPA FRANCORCHAMPS':'Spa', 'BAHRAIN INTERNATIONAL CIRCUIT':'Bahrain', 'FUJI SPEEDWAY':'Fuji', 'SHANGHAI INTERNATIONAL CIRCUIT':'Shanghai', 'CIRCUIT OF THE AMERICAS':'COTA',
    'AUTODROMO HERMANOS RODRIGUEZ':'Mexico', 'LE MANS 2018':'Le Mans', 'SPA FRANCORCHAMPS 2019':'Spa','LE MANS 2019':'Le Mans', 'BAHRAIN INTERNATIONAL CIRCUIT 2019': 'Bahrain',
    'BAHRAIN INTERNATIONAL CIRCUIT 2020':'Bahrain', 'AUTODROMO DO ALGARVE':'Portimao', 'AUTODROMO NAZIONALE DI MONZA':'Monza', 'BAHRAIN INTERNATIONAL CIRCUIT 6 HOURS':'Bahrain', 'BAHRAIN INTERNATIONAL CIRCUIT 8 HOURS':'Bahrain'})
    df['circuit'] = df['circuit'].str.lower() 
#     Replacing the long-form and/or year appended circuit names with the common-use short versions. Also changes all to lower-case


    df = df.drop('flag_at_fl', axis=1)
#     df in column only exists for season 2022, some value meanings can not be determined.

    df = df.dropna().reset_index(drop=True)
# Dropping NaN values to remove rows with missing data and resetting rthe index


    return df


In [6]:
data = fixing_kept_cols(data)

/tmp/ipykernel_1344/2701601627.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['circuit'] = df['circuit'].replace({'SPA FRANCORCHAMPS':'Spa', 'BAHRAIN INTERNATIONAL CIRCUIT':'Bahrain', 'FUJI SPEEDWAY':'Fuji', 'SHANGHAI INTERNATIONAL CIRCUIT':'Shanghai', 'CIRCUIT OF THE AMERICAS':'COTA',
/tmp/ipykernel_1344/2701601627.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['circuit'] = df['circuit'].str.lower()


In [7]:
def separate_df(df):
    cont_nums = df.select_dtypes('number')
    cont_nums = cont_nums.drop(['lap_improvement', 'crossing_finish_line_in_pit', 's1_improvement', 's2_improvement', 's3_improvement', 'year', 'round', 'driver_stint_no', 'class_position'], axis=1)
    
    disc_nums = df[['lap_improvement', 'crossing_finish_line_in_pit', 's1_improvement', 's2_improvement', 's3_improvement', 'year', 'round', 'driver_stint_no', 'class_position']]
    
    obs = data.select_dtypes(object).drop('rating', axis=1)

    
    
    
    target = df['rating']
    
    return cont_nums, disc_nums, obs, target

In [8]:
cont_nums, disc_nums, obs, target = separate_df(data)

In [9]:
def encode_obs(obs):
    class_one_hot = pd.get_dummies(obs['class'], prefix='class')
    circuit_one_hot = pd.get_dummies(obs['circuit'], prefix='circuit')
    
    ob_feats = pd.concat([class_one_hot, circuit_one_hot], axis=1)
    
    return ob_feats


In [10]:
ob_feats = encode_obs(obs)

In [11]:
def make_scaled_sets(df):
    from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
    
    s_scaler = StandardScaler()
    num_s_scal = s_scaler.fit_transform(df)
    num_s_scal = pd.DataFrame(num_s_scal)
    
    mm_scaler = MinMaxScaler()
    num_mm_scal = mm_scaler.fit_transform(df)
    num_mm_scal = pd.DataFrame(num_mm_scal)
    
    abs_scaler = MaxAbsScaler()
    num_abs_scal = abs_scaler.fit_transform(df)
    num_abs_scal = pd.DataFrame(num_abs_scal)
    
    
    return num_s_scal, num_mm_scal, num_abs_scal

In [12]:
num_s_scal, num_mm_scal, num_abs_scal = make_scaled_sets(cont_nums)